In [11]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

X_train = np.load('Xtrain_Classification1.npy') #6254 x 2352. 2352 = pixels x pixels x 3 (RGB). Training set is inbalanced, different number of samples for each class
y_train = np.load('ytrain_Classification1.npy') #6254. 1D vector
X_test = np.load('Xtest_Classification1.npy') #1764 x 2352. Has data from two distinct sourcesprint(X_train[0:10,0:2], y_train[0:200])

We should scaler down the values of the training data by dividing each value with 255 (highest value of "brightness" of pixel) since classifiers can be sensitive to feature scaling. In this case, we will standardize it.

In [12]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test) #apply same transformation to test data

We need to define possible values of alpha, the regularization term in the MLP classifier function that prevents overfitting.

In [13]:
def create_lambdas(begin, end, increment):
    """ Creates a list of hyperparameter values for cross-validation. """
    return [begin + i * increment for i in range(int((end - begin) / increment))]

In [14]:
alphas_list = create_lambdas(0.1, 10, 1)
lrate_init = create_lambdas(0.0001, 0.001, 0.0001)
parameters = {'alpha': alphas_list} #To add to GridSearchCV we need to create a dictionary

‘adam’ refers to a stochastic gradient-based optimizer proposed by Kingma, Diederik, and Jimmy Ba

In [15]:
classifier = MLPClassifier(solver= 'adam') #Keep default solver 'adam' since is better for larger datasets and is robust
grid_search = GridSearchCV(classifier, parameters) 

grid_search.fit(X_train, y_train)

best_estimator = grid_search.best_estimator_
print(best_estimator)

MLPClassifier(alpha=0.1)


/Users/joaodias/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Data needs to be reshaped. If N is number of samples np.reshape(X,(N,28,28,3))
1- Build
Prof says we should define layers sequentialy.
We need to define activation function (ReLU, tanh, sigmoid...)
In the last layer we define depending on what we want. For regression we use linear, for classification we use nonlinear (sigmoid, softmax). Softmax needs to have as many units as classes in dataset.

2-Train
    -Define cost function(MSE for regression, Categorical cross entropy - binary cross entropy)
    -Define optimizer (sgd, momentum, adaptive steps...) One very common is adam
    -Define initial learning rate
    -Nº epochs. If we use train in mini-batch we need to define batch size.
    -We split data into train and validation data. We should plot the evolution of the loss along the epochs for both train data and validation data. The lines have the same shape but the loss is normally higher for the validation data. NOTE: If loss starts to increase suddently at a certain epoch in validation data then we have an overfitted network. It's for this reason that we should use an early stopping callback (stops if loss starts growing after a certain epochs). 
    -For the batch size just choose one we are happy with and computer doesn't take a lot to run. Don't try to find the optimal value.

3 - Predictions
    w_hat = arg max(y_hat)
    